# Laboratorio de Desarrollo 1: De Notebook a Hugging Face Space

**Procesamiento del Habla e Introducción a los LLMs**

*Material creado por Matías Barreto, 2025*

---

## Objetivos del Laboratorio

Al finalizar este laboratorio, vas a poder:

1. Comprender la arquitectura de Hugging Face Hub y Spaces
2. Convertir código de Jupyter Notebook a scripts Python modulares
3. Crear una aplicación Gradio funcional para análisis de habla y texto
4. Deployar tu aplicación en Hugging Face Spaces
5. Integrar GitHub con Hugging Face para CI/CD
6. Configurar entornos virtuales para desarrollo local
7. Aplicar Vibe Engineering y VibeCoding para desarrollo efectivo con LLMs
8. Dominar el ciclo iterativo MVP para prototipado rápido

---

## Parte 0: Configuración de Entorno de Desarrollo

### Por qué Entornos Virtuales son Críticos

Antes de desarrollar en local, **siempre** hay que crear un entorno virtual. Esto evita conflictos de dependencias y mantiene tu sistema limpio.

**Problemas sin entorno virtual:**

```
Proyecto A necesita transformers 4.35.0
Proyecto B necesita transformers 4.20.0
→ CONFLICTO: uno de los dos va a fallar
```

**Solución: Entornos aislados**

```
venv_proyecto_A/
  └─ transformers 4.35.0

venv_proyecto_B/
  └─ transformers 4.20.0

→ Ambos funcionan sin interferencia
```

### Crear Entorno Virtual

**Opción 1: venv (nativo de Python)**

```bash
# Crear entorno virtual
python -m venv venv_habla_lab1

# Activar (Windows)
venv_habla_lab1\Scripts\activate

# Activar (Linux/Mac)
source venv_habla_lab1/bin/activate

# Verificar que estás en el entorno
(venv_habla_lab1) PS C:\tu-proyecto>
```

**Opción 2: conda (si usás Anaconda)**

```bash
# Crear entorno
conda create -n habla_lab1 python=3.10

# Activar
conda activate habla_lab1
```

### Instalar Dependencias en el Entorno

Una vez activado el entorno:

```bash
# Instalar paquetes necesarios
pip install transformers>=4.35.0 torch>=2.0.0 gradio>=4.0.0

# O desde requirements.txt
pip install -r requirements.txt
```

### Desactivar Entorno

```bash
# venv
deactivate

# conda
conda deactivate
```

### .gitignore para Entornos Virtuales

**Nunca** subir el entorno virtual a Git. Crear `.gitignore`:

```
# Entornos virtuales
venv/
venv_*/
env/
.venv/

# Conda
envs/

# Python
__pycache__/
*.pyc
*.pyo
.DS_Store

# IDEs
.vscode/
.idea/
```

---

## Parte 1: Hugging Face en Profundidad

### Qué es Hugging Face

**Hugging Face** es el "GitHub de la IA" - una plataforma que democratiza el acceso a:

- **Models**: Más de 500,000 modelos preentrenados (BERT, GPT, Whisper, etc.)
- **Datasets**: Más de 100,000 datasets públicos (Common Voice, LibriSpeech)
- **Spaces**: Aplicaciones ML deployadas gratuitamente
- **Documentación**: Cursos y tutoriales completos

### Qué es un Space

Un **Space** es una aplicación web hosteada gratuitamente que ejecuta:

- **Gradio**: Interfaces ML interactivas (lo usaremos hoy)
- **Streamlit**: Dashboards de datos
- **Static HTML**: Páginas web estáticas
- **Docker**: Contenedores personalizados

**Ventajas de Spaces:**

- Hosting gratuito
- Deploy automático desde GitHub
- GPU disponible (con límites)
- Compartible con URL pública
- Embebible en sitios web

---

## Parte 2: Anatomía de un Hugging Face Space

### Estructura de Archivos Obligatoria

```
mi-space-sentimientos/
├── app.py              # ARCHIVO PRINCIPAL - punto de entrada
├── requirements.txt    # DEPENDENCIAS - paquetes necesarios
├── README.md          # METADATOS - configuración del Space
├── .gitignore         # Archivos a ignorar
└── (opcional)
    ├── utils.py       # Funciones auxiliares
    └── examples/      # Archivos de ejemplo
```

### El Archivo app.py - Corazón del Space

```python
"""
app.py - Clasificador de Sentimientos para Transcripciones de Audio
Este archivo DEBE llamarse 'app.py' para que Hugging Face lo reconozca
"""

import gradio as gr
from transformers import pipeline

# 1. CARGAR MODELO (se ejecuta UNA VEZ al inicio)
print("Cargando modelo RoBERTuito...")
clasificador = pipeline(
    "sentiment-analysis",
    model="pysentimiento/robertuito-sentiment-analysis"
)
print("Modelo cargado exitosamente")

# 2. FUNCIÓN DE PROCESAMIENTO
def analizar_sentimiento(texto):
    """
    Analiza el sentimiento de una transcripción de audio.
    
    Args:
        texto: str - Transcripción del audio
    
    Returns:
        dict: Resultados en formato {label: score}
    """
    if not texto.strip():
        return {"Error": 1.0}
    
    resultado = clasificador(texto)[0]
    
    # Mapear etiquetas a español
    etiquetas = {
        "POS": "Positivo",
        "NEG": "Negativo",
        "NEU": "Neutral"
    }
    
    label = etiquetas.get(resultado['label'], resultado['label'])
    return {label: float(resultado['score'])}

# 3. INTERFAZ GRADIO
demo = gr.Interface(
    fn=analizar_sentimiento,
    inputs=gr.Textbox(
        label="Transcripción de Audio",
        placeholder="Pegá aquí la transcripción del audio...",
        lines=5
    ),
    outputs=gr.Label(
        num_top_classes=3,
        label="Sentimiento Detectado"
    ),
    title="Analizador de Sentimientos para Habla",
    description="""
    Analiza el sentimiento de transcripciones de audio usando RoBERTuito.
    Entrenado en español argentino (tweets).
    """,
    examples=[
        ["La charla estuvo excelente, aprendí un montón."],
        ["Qué garrón de entrevista, no me fue nada bien."]
    ]
)

# 4. LANZAR APLICACIÓN
if __name__ == "__main__":
    demo.launch()
```

### El Archivo requirements.txt

```txt
transformers>=4.35.0
torch>=2.0.0
gradio>=4.0.0
```

### El Archivo README.md

```markdown
---
title: Analizador de Sentimientos para Habla
emoji: 🎙️
colorFrom: green
colorTo: blue
sdk: gradio
sdk_version: 4.44.1
app_file: app.py
pinned: false
license: mit
---

# Analizador de Sentimientos para Transcripciones de Audio

Analiza el sentimiento de transcripciones usando RoBERTuito.
```

---

## Parte 3: De Notebook a Script Python

### Diferencias Fundamentales

| Aspecto | Jupyter Notebook | Script Python |
|---------|------------------|---------------|
| **Ejecución** | Celda por celda | Todo de una vez |
| **Estado** | Interactivo, mutable | Declarativo, reproducible |
| **Imports** | Pueden estar dispersos | Al inicio del archivo |
| **Outputs** | Visuales automáticos | Deben ser explícitos |
| **Debugging** | Fácil, paso a paso | Requiere logs/prints |
| **Producción** | No recomendado | Ideal |

### Proceso de Conversión - Ejemplo del Notebook 06

**ANTES - Notebook (celdas separadas):**

In [ ]:
# Celda 1: Instalación (NO VA EN EL SCRIPT)
# !pip install transformers torch

# Celda 2: Imports dispersos
from transformers import pipeline

# Celda 3: Cargar modelo
clasificador = pipeline(
    "sentiment-analysis",
    model="pysentimiento/robertuito-sentiment-analysis"
)

# Celda 4: Prueba interactiva (NO VA EN EL SCRIPT)
texto = "La charla estuvo excelente"
resultado = clasificador(texto)
print(resultado)

**DESPUÉS - Script Python (app.py):**

```python
"""
app.py - Analizador de Sentimientos
Convertido desde notebook 06_Transformers_Preentrenados_HuggingFace.ipynb
"""

# 1. IMPORTS - Todos al inicio
import gradio as gr
from transformers import pipeline

# 2. CONFIGURACIÓN - Constantes globales
MODEL_NAME = "pysentimiento/robertuito-sentiment-analysis"
ETIQUETAS = {"POS": "Positivo", "NEG": "Negativo", "NEU": "Neutral"}

# 3. CARGA DEL MODELO - Una sola vez
print("Cargando modelo...")
clasificador = pipeline("sentiment-analysis", model=MODEL_NAME)
print("Modelo cargado")

# 4. FUNCIÓN DE PROCESAMIENTO - Lógica principal
def analizar_sentimiento(texto):
    """Analiza sentimiento y retorna resultado formateado."""
    try:
        if not texto.strip():
            return {"Error": 1.0}
        
        resultado = clasificador(texto)[0]
        label = ETIQUETAS.get(resultado['label'], resultado['label'])
        return {label: float(resultado['score'])}
    except Exception as e:
        return {"Error": str(e)}

# 5. INTERFAZ GRADIO - Definición de UI
demo = gr.Interface(
    fn=analizar_sentimiento,
    inputs=gr.Textbox(label="Texto", lines=3),
    outputs=gr.Label(num_top_classes=3),
    title="Analizador de Sentimientos"
)

# 6. PUNTO DE ENTRADA
if __name__ == "__main__":
    demo.launch()
```

### Checklist de Conversión

- Eliminar `!pip install` y mover a `requirements.txt`
- Todos los imports al inicio del archivo
- Cargar modelo UNA VEZ (fuera de funciones)
- Funciones con docstrings claros
- Manejo de errores con try/except
- Interfaz Gradio reemplaza inputs interactivos
- `if __name__ == "__main__"` para punto de entrada

---

## Parte 4: Vibe Engineering - Desarrollo con IA Generativa

### Qué es Vibe Engineering

**Vibe Engineering** es una metodología de desarrollo que aprovecha LLMs como asistentes de codificación. A diferencia del desarrollo tradicional, priorizás:

- **Iteración rápida** sobre planificación exhaustiva
- **Conversación natural** con LLMs como compañeros de trabajo
- **Experimentación** sobre documentación detallada previa
- **Prototipado rápido** para validar ideas

**Comparación:**

```
Desarrollo tradicional:
  Diseño → Documentación → Implementación → Testing
  (Días/semanas)

Vibe Engineering:
  Idea → Prompt al LLM → Código → Probar → Iterar
  (Minutos/horas)
```

### El Ciclo Iterativo MVP

El desarrollo con IA generativa sigue este ciclo:

```
1. MVP o Feature
   ↓
2. Test
   ↓
3. Error
   ↓
4. Debug
   ↓
5. Checkpoint
   ↓
   ← Volver al inicio con nueva feature
```

**Características del ciclo:**

- **Rápido**: Cada iteración toma minutos, no horas
- **Incremental**: Construís sobre lo que ya funciona
- **Checkpoint-driven**: Validás cada paso antes de avanzar
- **Colaborativo**: El LLM es tu par de programación

### Ciclo de Desarrollo de IA Generativa (6 Pasos)

**Paso 1: Empezá con una idea**

Definí qué querés construir de forma simple:
```
"Quiero analizar el sentimiento de transcripciones de audio"
```

**Paso 2: Construí un prototipo rápido usando Python y Gradio**

Pedile al LLM que genere el código base:
```
Prompt: "Creame una app Gradio que analice sentimientos de texto
usando el modelo pysentimiento/robertuito-sentiment-analysis"
```

**Paso 3: Probá el prototipo**

Ejecutá localmente y probá con casos reales:
```bash
python app.py
# Probá con varios textos
```

**Paso 4: Compartí el prototipo y obtene retroalimentación**

Mostralo a compañeros, profesores, usuarios potenciales. Anotá:
- Qué funciona bien
- Qué falla
- Qué falta

**Paso 5: Mejorá el prompt, el código o la interfaz basándote en lo aprendido**

Iterá con el LLM:
```
Prompt: "El modelo funciona pero necesito agregar análisis de emociones.
¿Qué modelo me recomendás y cómo lo integro sin romper lo que ya tengo?"
```

**Paso 6: Decidí si vale la pena convertirlo en una aplicación completa**

Evaluá:
- ¿El prototipo resuelve el problema?
- ¿Hay interés real?
- ¿Vale la pena pulirlo y deployarlo?

Si sí, volvé al paso 2 con nuevas features.

### Principios para Llegar al MVP

Cuando trabajes con LLMs para desarrollar:

**1. Dale al LLM solo la información relevante al MVP**

```
Mal prompt:
"Quiero una app de análisis de sentimientos que use 5 modelos diferentes,
con base de datos, autenticación, gráficos, exportación a PDF..."

Buen prompt:
"Creame una app Gradio simple que analice sentimientos de texto usando
RoBERTuito. Solo entrada de texto y salida de clasificación."
```

**2. Empezá pequeño y construí hacia arriba**

```
Iteración 1: Clasificación básica
Iteración 2: Agregar ejemplos
Iteración 3: Mejorar UI
Iteración 4: Agregar segunda pestaña para análisis en lote
```

**3. Proporcioná contexto fundacional y detalles importantes**

```
Buen prompt:
"Estoy trabajando en una app de análisis de habla. Ya tengo un clasificador
de sentimientos funcionando con RoBERTuito en app.py que carga el modelo
una sola vez al inicio.

Necesito agregar análisis de emociones. El código actual es:
[pegar código]

¿Cómo integro pysentimiento/robertuito-emotion-analysis manteniendo
la misma arquitectura?"
```

---

## Parte 5: VibeCoding - Los 4 Tipos de Pensamiento

Cuando desarrollás código (con o sin LLMs), aplicás cuatro tipos de pensamiento complementarios:

### 1. Pensamiento Lógico

**Pregunta clave:** ¿Qué es el juego?

En programación: ¿Qué hace este código? ¿Cuál es el flujo básico?

```python
# Ejemplo: Entender la lógica
def analizar_sentimiento(texto):
    # Entrada: texto
    # Proceso: pasar por modelo
    # Salida: clasificación
    resultado = clasificador(texto)
    return procesar(resultado)
```

**Aplicación con LLMs:**
```
Prompt: "Explicame paso a paso qué hace esta función:
[pegar código]"
```

### 2. Pensamiento Analítico

**Pregunta clave:** ¿Cómo juego el juego?

En programación: ¿Por qué funciona así? ¿Qué decisiones de diseño se tomaron?

```python
# Ejemplo: Analizar decisiones de diseño
clasificador = pipeline("sentiment-analysis")  # ¿Por qué pipeline?
# Respuesta: Pipeline abstrae tokenización, preprocessing, postprocessing

# ¿Por qué cargar modelo al inicio y no en la función?
# Respuesta: Evitar recargar el modelo en cada predicción (muy lento)
```

**Aplicación con LLMs:**
```
Prompt: "¿Por qué este código carga el modelo fuera de la función?
¿Qué pasaría si lo cargo adentro?"
```

### 3. Pensamiento Computacional

**Pregunta clave:** ¿Cuáles son los patrones detrás del juego?

En programación: ¿Qué patrones y optimizaciones puedo aplicar?

```python
# Ejemplo: Identificar patrones ineficientes

# INEFICIENTE - Patrón anti-pattern
def procesar_lote(textos):
    resultados = []
    for texto in textos:
        modelo_temp = pipeline("...")  # Recarga modelo cada vez
        resultados.append(modelo_temp(texto))
    return resultados

# EFICIENTE - Patrón singleton para modelo
modelo = pipeline("...")  # Cargar UNA vez
def procesar_lote(textos):
    return [modelo(t) for t in textos]  # Reutilizar modelo
```

**Aplicación con LLMs:**
```
Prompt: "Este código funciona pero es lento. ¿Qué patrones de optimización
puedo aplicar?
[pegar código]"
```

### 4. Pensamiento Procedimental

**Pregunta clave:** ¿Cómo sobresalgo en el juego?

En programación: ¿Cuál es el paso a paso exacto? ¿Cómo lo ejecuto bien?

```python
# Ejemplo: Descomponer en pasos procedimentales
def app_completa():
    # Paso 1: Cargar modelo
    print("Paso 1: Cargando modelo...")
    modelo = cargar_modelo()
    print("Modelo cargado")
    
    # Paso 2: Definir función de procesamiento
    print("Paso 2: Definiendo función...")
    def procesar(texto):
        return modelo(texto)
    
    # Paso 3: Crear interfaz
    print("Paso 3: Creando interfaz...")
    interfaz = crear_gradio(procesar)
    
    # Paso 4: Lanzar
    print("Paso 4: Lanzando aplicación...")
    interfaz.launch()
```

**Aplicación con LLMs:**
```
Prompt: "Dame un plan paso a paso para convertir este notebook a un Space.
Numerá cada paso y decime qué archivos crear."
```

### Pirámide de Complejidad

Los cuatro tipos de pensamiento se construyen uno sobre otro:

```
                  Procedimental
                (Cómo sobresalir)
               /                 \
        Computacional              \
      (Patrones y optimización)     \
         /                           \
    Analítico                         \
  (Por qué funciona así)               \
       /                                \
   Lógico ――――――――――――――――――――――――――――――
 (Qué hace)
```

**En la práctica:**

1. Empezás con **Lógico**: "¿Qué hace este código?"
2. Avanzás a **Analítico**: "¿Por qué está estructurado así?"
3. Aplicás **Computacional**: "¿Cómo lo optimizo?"
4. Dominás con **Procedimental**: "¿Cuál es el mejor workflow?"

---

## Parte 6: Debugging Sistemático y Contexto

### El Debugging es Aburrido pero Necesario

Como dice el material: "Es aburrido, pero podés hacerlo divertido."

El mejor debugging es **metódico y exhaustivo**:

```
Problema: La lámpara no funciona
   ↓
¿Está enchufada? → No → Enchufar lámpara
   ↓ Sí
¿La bombilla está quemada? → Sí → Reemplazar bombilla
   ↓ No
Reparar lámpara
```

### Estrategia de Debugging con LLMs

**Objetivos:**
- Entender cómo funciona tu app
- Entender dónde está el error

**Preguntas clave:**
- ¿Cómo llegás a la raíz del problema?
- ¿Cómo le contás al LLM qué está mal?

**Método paso a paso:**

```python
# 1. Checkpoint: Modelo carga correctamente
print("CHECKPOINT 1: Cargando modelo...")
modelo = pipeline("sentiment-analysis")
print("Modelo cargado exitosamente")

# 2. Checkpoint: Función procesa correctamente
print("CHECKPOINT 2: Probando función...")
test_texto = "Esto es una prueba"
resultado = analizar(test_texto)
assert isinstance(resultado, dict), "Debe retornar dict"
print("Función funciona correctamente")

# 3. Checkpoint: Interfaz se crea
print("CHECKPOINT 3: Creando interfaz...")
demo = gr.Interface(...)
print("Interfaz creada exitosamente")

# 4. Checkpoint: App lanza localmente
print("CHECKPOINT 4: Lanzando app...")
demo.launch()
print("App lanzada en http://localhost:7860")
```

**Comunicar errores al LLM:**

```
Mal prompt:
"No funciona, ayuda"

Buen prompt:
"Tengo este error en el CHECKPOINT 2:

Error:
TypeError: 'NoneType' object is not subscriptable
  File "app.py", line 15, in analizar_sentimiento
    label = resultado['label']

Código:
[pegar código relevante]

¿Qué está pasando?"
```

### Contexto: La Ventana de Contexto

**Contexto** en el mundo de LLMs significa: **la cantidad de tokens que un LLM puede procesar en un momento dado**.

**Context window (ventana de contexto):**
- GPT-4: 128,000 tokens (~96,000 palabras)
- Claude 3.5: 200,000 tokens (~150,000 palabras)
- Llama 3: 8,000 tokens (~6,000 palabras)

**El contexto puede ser tu prompt, pero también:**
- Código previo que le mostraste
- Conversación anterior
- Documentación que le pasaste
- Errores anteriores

**Estrategia de contexto efectivo:**

```
Contexto inicial (una vez):
"Estoy construyendo una app Gradio para análisis de sentimientos de
transcripciones de audio. Uso Python 3.10, transformers 4.35,
y el modelo RoBERTuito. La app se va a deployar en Hugging Face Spaces.

La estructura es:
- app.py: código principal
- requirements.txt: dependencias
- README.md: metadatos

Arquitectura: Cargamos el modelo UNA vez al inicio, luego definimos
funciones de procesamiento, y finalmente creamos la interfaz Gradio."

Luego, en cada prompt:
"[Tu pregunta específica sin repetir todo el contexto]"
```

El LLM mantiene el contexto de la conversación, así que no tenés que repetir todo cada vez.

---

## Parte 7: Desarrollo Local con Entorno Virtual

### Workflow Completo

```bash
# 1. Crear directorio del proyecto
mkdir analizador-sentimientos
cd analizador-sentimientos

# 2. Crear entorno virtual
python -m venv venv

# 3. Activar entorno (Windows)
venv\Scripts\activate

# 3. Activar entorno (Linux/Mac)
source venv/bin/activate

# 4. Verificar que estás en el entorno
(venv) $  # ← El prompt cambia

# 5. Instalar dependencias
pip install transformers torch gradio

# 6. Crear requirements.txt
pip freeze > requirements.txt

# 7. Crear archivos del proyecto
touch app.py README.md .gitignore

# 8. Probar la app localmente
python app.py
# → Abre http://localhost:7860

# 9. Cuando termines de trabajar
deactivate
```

### Estructura del Proyecto

```
analizador-sentimientos/
├── venv/                  # ← Entorno virtual (NO subir a Git)
├── app.py                 # ← Código principal
├── requirements.txt       # ← Dependencias
├── README.md             # ← Documentación
└── .gitignore            # ← Ignorar venv/
```

---

## Parte 8: Deploy a Hugging Face Spaces

### Preparación

Antes de deployar, verificar:

```bash
# Probar localmente
python app.py
# ✓ Funciona sin errores

# Verificar archivos
ls
# ✓ app.py existe
# ✓ requirements.txt existe
# ✓ README.md existe
```

### Crear Space en Hugging Face

1. Ir a https://huggingface.co/new-space
2. Elegir nombre: `analizador-sentimientos-habla`
3. Elegir SDK: **Gradio**
4. Visibility: **Public**
5. Click **Create Space**

### Conectar Git y Pushear

```bash
# Inicializar repo (si no lo hiciste)
git init

# Agregar archivos
git add app.py requirements.txt README.md .gitignore

# Commit inicial
git commit -m "feat: Analizador de sentimientos inicial"

# Conectar con Hugging Face
git remote add origin https://huggingface.co/spaces/TU_USUARIO/analizador-sentimientos-habla

# Configurar branch principal
git branch -M main

# Push
git push -u origin main
```

### Verificar Deploy

1. Ir a tu Space: `https://huggingface.co/spaces/TU_USUARIO/analizador-sentimientos-habla`
2. Ver logs en la pestaña "Logs"
3. Esperar 2-3 minutos para el build
4. Probar la aplicación
5. Deploy exitoso

### Updates Posteriores

```bash
# Modificar app.py
# ...

# Commit y push
git add app.py
git commit -m "fix: Mejorar manejo de errores"
git push

# → El Space se actualiza automáticamente
```

---

## Parte 9: Ejercicio Práctico Guiado

### Objetivo

Crear un Space que analice el sentimiento de transcripciones de audio usando el modelo **RoBERTuito**, aplicando Vibe Engineering.

### Paso a Paso

#### Paso 1: Crear entorno virtual

```bash
mkdir analizador-sentimientos-habla
cd analizador-sentimientos-habla
python -m venv venv
venv\Scripts\activate  # Windows
# source venv/bin/activate  # Linux/Mac
```

#### Paso 2: Instalar dependencias

```bash
pip install transformers torch gradio
pip freeze > requirements.txt
```

#### Paso 3: Usar LLM para generar app.py

**Prompt al LLM (Claude, ChatGPT, etc.):**

```
Creame un archivo app.py para una aplicación Gradio que:
1. Analice sentimientos de texto usando pysentimiento/robertuito-sentiment-analysis
2. Cargue el modelo UNA vez al inicio
3. Tenga una función que reciba texto y devuelva el sentimiento en español
4. Use gr.Interface con input de texto y output de Label
5. Incluya 2 ejemplos en español argentino

El código debe ser simple, bien comentado y seguir buenas prácticas.
```

#### Paso 4: Crear README.md

```markdown
---
title: Analizador de Sentimientos para Habla
emoji: 🎙️
colorFrom: green
colorTo: blue
sdk: gradio
sdk_version: 4.44.1
app_file: app.py
pinned: false
license: mit
---

# Analizador de Sentimientos para Transcripciones

Analiza el sentimiento de transcripciones de audio usando RoBERTuito.
Entrenado en español argentino.

## Uso
1. Pega la transcripción del audio
2. Click en Submit
3. Ve el resultado
```

#### Paso 5: Crear .gitignore

```
venv/
venv_*/
env/
.venv/
__pycache__/
*.pyc
.vscode/
.idea/
.DS_Store
```

#### Paso 6: CHECKPOINT - Probar localmente

```bash
python app.py
# Abrir http://localhost:7860
# Probar con textos de ejemplo
# Verificar que funciona
```

#### Paso 7: Deploy a Hugging Face

```bash
git init
git add .
git commit -m "feat: Analizador de sentimientos inicial"
git remote add origin https://huggingface.co/spaces/TU_USUARIO/analizador-sentimientos-habla
git branch -M main
git push -u origin main
```

#### Paso 8: CHECKPOINT - Verificar deploy

1. Ver logs en Hugging Face
2. Esperar build
3. Probar app en URL pública
4. Si hay errores, copiar el error y pedirle ayuda al LLM

---

## Parte 10: Ejercicios Complementarios

### Ejercicio 1: Iteración - Agregar Más Ejemplos

**Prompt sugerido al LLM:**
```
Mi app.py actual tiene 2 ejemplos. Quiero agregar 3 ejemplos más de:
- Una entrevista laboral positiva
- Una reseña de podcast negativa  
- Una transcripción de llamada neutral

Todos en español argentino coloquial.
```

### Ejercicio 2: Nueva Feature - Análisis en Lote

**Prompt sugerido al LLM:**
```
Tengo esta app de análisis de sentimientos. Quiero agregar una segunda
pestaña (Tab) donde pueda pegar múltiples transcripciones (una por línea)
y obtener el análisis de todas a la vez.

Código actual:
[pegar app.py]

¿Cómo lo hago usando gr.Blocks con gr.Tab?
```

### Ejercicio 3: Comparación de Modelos

**Prompt sugerido al LLM:**
```
Quiero crear una interfaz que compare BETO vs RoBERTuito lado a lado.
El usuario ingresa un texto y ve los resultados de ambos modelos.

Necesito:
1. Cargar ambos modelos al inicio
2. Una función que procese con ambos
3. Mostrar resultados en dos columnas

¿Cómo estructuro el código?
```

---

## Recursos Adicionales

### Documentación

- [Gradio Docs](https://www.gradio.app/docs)
- [Hugging Face Spaces](https://huggingface.co/docs/hub/spaces)
- [Transformers Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines)
- [Vibe Engineering - Simon Willison](https://simonwillison.net/2025/Oct/7/vibe-engineering/)

### Modelos Recomendados para Habla y Transcripciones

- **Sentimientos**: `pysentimiento/robertuito-sentiment-analysis`
- **Emociones**: `pysentimiento/robertuito-emotion-analysis`
- **NER**: `mrm8488/bert-spanish-cased-finetuned-ner`
- **Transcripción**: `openai/whisper-large-v3`

### Herramientas

- **VS Code**: Editor recomendado
- **Claude / ChatGPT**: Asistentes IA para código
- **Git**: Control de versiones

---

## Checklist Final

### Pre-Deploy

- Entorno virtual creado y activado
- app.py funciona localmente
- requirements.txt completo
- README.md con metadatos YAML
- .gitignore excluye venv/
- Probado con varios textos de ejemplo

### Deploy

- Space creado en Hugging Face
- Git configurado y conectado
- Push realizado sin errores
- Build exitoso (revisar logs)
- Aplicación funcional en la URL pública

### Post-Deploy

- Probar todas las funcionalidades
- Verificar ejemplos
- Compartir con compañeros
- (Opcional) Conectar con GitHub para CI/CD

---

## Conclusión

Felicitaciones por completar el Laboratorio 1 de Desarrollo.

### Lo que aprendiste:

- Configurar entornos virtuales para desarrollo local
- Convertir notebooks a aplicaciones de producción
- Crear interfaces Gradio interactivas
- Deployar a Hugging Face Spaces
- Aplicar Vibe Engineering para desarrollo con LLMs
- Usar los 4 tipos de pensamiento (VibeCoding)
- Debugging sistemático con checkpoints
- Ciclo iterativo MVP
- Trabajar con modelos de NLP en español

### Próximos pasos:

1. Experimentar con otros modelos de análisis de habla
2. Integrar transcripción automática (Whisper)
3. Agregar visualizaciones de resultados
4. Escalar a análisis en tiempo real

---

**Ahora podés crear tus propias aplicaciones de procesamiento del habla y compartirlas con el mundo usando Vibe Engineering como metodología de desarrollo.**